# Planeación campaña 2017


## 1. Introducción y Objetivos

#### Llegar a más jugadores revisando los juegos exitosos

En qué tipo de juegos debemos enfocar nuestra producción?, son exitos del momento?, cuál fue el mejor año para los videojuegos?, los jugadores de Norte America, Europa, Japón y otras partes del mundo tienen los mismos gustos de juegos?.

En éste artículo deseamos exponer qué nos dicen los datos acerca de los hábitos de consumo en cada lugar y cómo focalizar nuestros esfuerzos a crear contenido que sea rentable en todo el mundo.

### 1.1 Descripción del proyecto

La tienda Ice vende videojuegos por todo el mundo, en este proyecto vamos a identificar si un juego tiene éxito para detectar proyectos prometedores y planificar campañas. 

### 1.2 Objetivos
Crearemos perfiles de usuario para cada región, NA, UE, JP y en base a ello determinaremos las cinco plataformas y géneros principales, y si las clasificaciones afectan las ventas individuales.




## 2. Exploración y Limpieza de Datos




### 2.1 Carga de datos

Cargamos los datos dentro de un DataFrame para poder manipularlos y revisamos cómo están conformados los datos.

In [55]:
import pandas as pd

#------------Leemos los datos del archivo games-------------
df = pd.read_csv('datasets/games.csv')

#Definimos una función de apoyo para poder explorar rápidamente los datos en cada DF

def verificar(df):
    """Función para verificar errores comunes Data frames"""
    print("=== VERIFICACIÓN ===")

    # 1. Información básica
    print(f"Forma: {df.shape}")
    print(f"Columnas: {list(df.columns)}")

    # 2. Valores nulos
    print(f"\nValores nulos:\n{df.isnull().sum()}")

    # 3. Tipos de datos
    print(f"\nTipos de datos:\n{df.dtypes}")
    
    # 4. Duplicados
    print(f"\nFilas duplicadas: {df.duplicated().sum()}")
    
    # 5. Estadísticas básicas
    print(f"\nEstadísticas descriptivas:")
    print(df.describe())


display(df.head())
df.info()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


### 2.2 Limpieza inicial

En esta sección realizaremos la limpieza inicial de los datos para:

* Estandarizar nombres de columnas
* Limpiar valores de texto (minúsculas, espacios)

In [56]:

#---------Columnas a minúsculas y sin espacios al inicio y al final--------
df.columns = df.columns.str.lower().str.strip()

#-----------------Columnas de datos String con minusculas y sin espacios al inicio ni al final--------------
df['name'] = df['name'].str.lower().str.strip()
df['platform'] = df['platform'].str.lower().str.strip()
df['genre'] = df['genre'].str.lower().str.strip()

#-----------------Columnas separadas -------------------

name = df['name']
platform = df['platform']
year = df['year_of_release']
genre = df['genre']
na_sales = df['na_sales']



Aquí revisamos los tipos de datos en las columnas y se cambian a los tipos correctos.


    

In [57]:
# ---------------Los datos de tipo entero asignados correctamente ----------------
df['year_of_release'] = pd.to_numeric(df['year_of_release'], errors='coerce').astype('Int64')
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce').astype('float64')

Por último confirmemos los cambios que realizamos en el data frame


### 2.3 Tratamiento de valores faltantes y duplicados

En este apartado validamos los datos para tratar los faltantes, evitando así desviaciones.

Para empezar verificamos para cada columna los datos vacíos.

In [78]:
print(df.isnull().sum())
print(df['name'].unique())

name                  0
platform              0
year_of_release       0
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8576
user_score         9123
rating             6764
total_sales           0
dtype: int64
['wii sports' 'super mario bros.' 'mario kart wii' ...
 'woody woodpecker in crazy castle 5' 'lma manager 2007'
 'haitaka no psychedelica']


Comenzamos con la columna de nombres, son pocos, así que podemos filtrar para revisar los valores.

In [59]:
name_na = df[df['name'].isnull()]
print(name_na)

      name platform  year_of_release genre  na_sales  eu_sales  jp_sales  \
659    NaN      gen             1993   NaN      1.78      0.53      0.00   
14244  NaN      gen             1993   NaN      0.00      0.00      0.03   

       other_sales  critic_score  user_score rating  
659           0.08           NaN         NaN    NaN  
14244         0.00           NaN         NaN    NaN  


Al parecer coinciden las filas sin nombre y sin género. Retiremoslo de los valores del DataFrame

In [60]:
df.dropna(subset = 'name', inplace = True)

Veamos si podemos tratar los datos de los años faltantes de la misma forma. Para ello identifiquemos la relación que tienen los datos vacios con respecto a los datos faltantes.

In [61]:
#-------------% de faltantes ----------------
faltantes = df['year_of_release'].isna().mean() * 100
print(f'Datos faltantes en year_of_release: {faltantes}%')

#------------Distribución de los datos---------------------
print(f'La media de los datos es: {df['year_of_release'].mean()} y la mediana es: {df['year_of_release'].median()}')

Datos faltantes en year_of_release: 1.6095255190570215%
La media de los datos es: 2006.4862563853078 y la mediana es: 2007.0


Usaremos la mediana para no sesgar los datos.

In [62]:
df['year_of_release'] = df['year_of_release'].fillna(df['year_of_release'].median()).astype('Int64')

Hagamos una operación semejante en las columnas critic_score y user_score.

In [63]:
#------% de faltalntes en la columna critic_score--------------
faltantes = df['critic_score'].isna().mean() * 100

print(f'Porcentaje de datos faltantes en critic_score: {faltantes}')

#--------Distribucion de los datos media y mediana

print(f'La media de los datos para esta columna es: {df['critic_score'].mean()} y  la mediana de los datos es: {df['critic_score'].median()}')
print()
#------% de faltalntes en la columna user_score--------------
faltantes = df['user_score'].isna().mean() * 100

print(f'Porcentaje de datos faltantes en user_score: {faltantes}')

#--------Distribucion de los datos media y mediana

print(f'La media de los datos para esta columna es: {df['user_score'].mean()} y  la mediana de los datos es: {df['user_score'].median()}')
print()


Porcentaje de datos faltantes en critic_score: 51.313348890085564
La media de los datos para esta columna es: 68.96767850559173 y  la mediana de los datos es: 71.0

Porcentaje de datos faltantes en user_score: 54.58625022437623
La media de los datos para esta columna es: 7.125046113306982 y  la mediana de los datos es: 7.5



Dado que no todos los títulos de videojuegos han sido calificados, tenemos bastantes datos faltantes.

Adicional a esto, no podemos tratar los datos de la misma manera y no podemos tratar el conjunto de datos de manera sencilla. 

En cambio, podemos separar las filas que contienen datos completos de las que no tienen para que podamos analizarlas aunque de forma separada y así no sesgar los datos con información poco certera.


In [64]:
#-----------------Separamos las filas que contienen resultados de las que no. --------------

user_score_notna = df[df['user_score'].notna()]
critic_score_notna = df[df['critic_score'].notna()]
rating_notna = df[df['rating'].notna()]

Vemos que los datos que hemos obtenido, en algunas filas coinciden pero en otras son complemento, por ello es mejor tratar cada columna como una un independiente.

#### Agreguemos datos útiles.

Es de utilidad validar cuales son las ventas totales para cada juego, por ello agregamos la columna ventas totales.

In [68]:
df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales']

## 3. Análisis Exploratorio

### Juegos lanzados por año.

Analicemos los datos de cada periodo para encontrar valores significativos.

Hagamos una división por año y cuantos juegos se lanzaron y año y ventas.

In [ ]:
year_group_count = df.groupby('year_of_release')['name'].count()
year_group_sales = df.groupby('year')

display(year_group_count)


year_of_release
1980       9
1981      46
1982      36
1983      17
1984      14
1985      14
1986      21
1987      16
1988      15
1989      17
1990      16
1991      41
1992      43
1993      60
1994     121
1995     219
1996     263
1997     289
1998     379
1999     338
2000     350
2001     482
2002     829
2003     775
2004     762
2005     939
2006    1006
2007    1466
2008    1427
2009    1426
2010    1255
2011    1136
2012     653
2013     544
2014     581
2015     606
2016     502
Name: year_of_release, dtype: Int64

In [ ]:




### 3.1 Distribución por plataformas
### 3.2 Tendencias por año
### 3.3 Análisis de géneros

## 4. Conclusiones y Recomendaciones